# Tarea Desafío KNN


**Desarrollado por:** Orlando Patricio Chacón Molina

**Fecha de creación:** 2022-02-06

**Fecha de actualización:** 2022-02-06

In [1]:
#Importar librerías
import pandas as pd
import numpy as np
# Para escalar datos
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split


In [2]:
filename = 'data/abalone.data'
df = pd.read_csv(filename, header=None)
df.head()

,0,1,2,3,4,5,6,7,8
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [3]:
"""
Sex		        nominal			M, F, and I (infant)
Length		    continuous	mm	Longest shell measurement
Diameter	    continuous	mm	perpendicular to length
Height		    continuous	mm	with meat in shell
Whole weight	continuous	grams	whole abalone
Shucked weight	continuous	grams	weight of meat
Viscera weight	continuous	grams	gut weight (after bleeding)
Shell weight	continuous	grams	after being dried
Rings		    integer			+1.5 gives the age in years
"""
df.columns=['Sex','Length','Diameter','Height','WholeEeight','ShuckedWeight','isceraWeight','ShellWeight','Rings']
df.head()



,Sex,Length,Diameter,Height,WholeEeight,ShuckedWeight,isceraWeight,ShellWeight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4177 entries, 0 to 4176
Data columns (total 9 columns):
Sex              4177 non-null object
Length           4177 non-null float64
Diameter         4177 non-null float64
Height           4177 non-null float64
WholeEeight      4177 non-null float64
ShuckedWeight    4177 non-null float64
isceraWeight     4177 non-null float64
ShellWeight      4177 non-null float64
Rings            4177 non-null int64
dtypes: float64(7), int64(1), object(1)
memory usage: 293.8+ KB


In [5]:
df.describe()

,Length,Diameter,Height,WholeEeight,ShuckedWeight,isceraWeight,ShellWeight,Rings
count,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000
mean,0.523992,0.407881,0.139516,0.828742,0.359367,0.180594,0.238831,9.933684
std,0.120093,0.099240,0.041827,0.490389,0.221963,0.109614,0.139203,3.224169
min,0.075000,0.055000,0.000000,0.002000,0.001000,0.000500,0.001500,1.000000
25%,0.450000,0.350000,0.115000,0.441500,0.186000,0.093500,0.130000,8.000000
50%,0.545000,0.425000,0.140000,0.799500,0.336000,0.171000,0.234000,9.000000
75%,0.615000,0.480000,0.165000,1.153000,0.502000,0.253000,0.329000,11.000000
max,0.815000,0.650000,1.130000,2.825500,1.488000,0.760000,1.005000,29.000000


In [6]:
df.groupby('Rings')['Length'].count().to_frame()

,Length
Rings,
1,1
2,1
3,15
4,57
5,115
6,259
7,391
8,568
9,689


## Estimación de la edad utilizando KNN(Regresión)

La edad del abulón se calcula de acuerdo al número de anillos:

$Edad = Rings +1.5$

In [54]:
X = df.loc[:,['Length','Diameter','Height','WholeEeight','ShuckedWeight','isceraWeight','ShellWeight']]
y = df.loc[:,['Rings']]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state=9993)

# Regresor KNN
neigh = KNeighborsRegressor(n_neighbors=2)
neigh.fit(X_train, y_train)

# El número de 'Rings' + 1.5 es la edad, por lo tanto habrá que sumar 1.5 a las predicciones
TestPredDf = pd.DataFrame(y_test)
TestPredDf['RingsPred']=neigh.predict(X_test)
TestPredDf['EdadPred']=neigh.predict(X_test)

# Se suma 1.5 a todas las predicciones y se pronostica la edad del abulón
TestPredDf['EdadPred'] = TestPredDf['EdadPred'].apply(lambda x: x+1.5)
TestPredDf.head()


,Rings,RingsPred,EdadPred
1619,10,10.0,11.5
59,7,10.5,12.0
3222,18,14.0,15.5
3612,10,11.5,13.0
1887,9,11.0,12.5


## Estimación del sexo utilizando KNN (Clasificación)

In [56]:
X = df.loc[:,['Length','Diameter','Height','WholeEeight','ShuckedWeight','isceraWeight','ShellWeight',]]
y = df.loc[:,['Sex']]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state=9993)

# Clasificador KNN
neigh = KNeighborsClassifier(n_neighbors=2)
neigh.fit(X_train, y_train)

# El número de 'Rings' + 1.5 es la edad, por lo tanto habrá que sumar 1.5 a las predicciones
TestPredDf = pd.DataFrame(y_test)
TestPredDf['SexPred']=neigh.predict(X_test)

# Se suma 1.5 a todas las predicciones y se pronostica la edad del abulón
TestPredDf.head()

C:\ProgramData\Anaconda3\envs\env36\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  import sys


,Sex,SexPred
1619,I,I
59,F,F
3222,M,F
3612,M,F
1887,I,I
